In [2]:
from config import OPENAI_KEY
from openai import OpenAI
import dotenv
dotenv.load_dotenv()

True

In [3]:
import os
client = OpenAI(api_key=os.getenv("OPENAI_KEY"))

In [4]:
# This is the new variant of gpt models, the communication is chat based.i.e questions/prompt and responses
def generate_text_with_chat(prompt):
    resp = client.chat.completions.create(
        model = 'gpt-4.1-nano',
        messages = [
            {
                "role": 'system',
                'content': [
                    {"type": "text", "text": 'you will be provided text and your responsible to fetch any personal identifiable content.'}
                ]
                
            },
            {
                'role': 'user',
                'content': [
                    {"type": "text", "text": "Mary went to Joyland school with a little lamb named blackie following her"}
                ],
            },
            {
                "role": 'assistant',
                'content': [
                    {"type": "text", "text": 'Mary is a girl, Joyland school is a place, blackie is the name of a sheep'}
                ]
                
            },
            {
                'role': 'user',
                'content': [
                    {"type": "text", "text": prompt}
                ],
            },
        ],
        temperature=0.1,
        max_tokens=60,
    )
    return resp

In [5]:
# Older style of gpt models, this is more like a sentence completion and not a chat based communication
def generate_text(prompt):
    resp = client.completions.create(
        model='davinci-002',
        prompt=prompt,
        max_tokens=90, # Based on the tokens setup, tokens will be used and the response's length will vary, proportionally (Ex: 55)
        temperature=0. # This means how random answers can be, means not random and 1 means very random (Ex: 0.4)
    )
    return resp

In [62]:
generated_text = generate_text('how do satellites maintain orientation?')
print(generated_text.choices[0].text)

 
- How to deal with a nosy fellow graduate student? 
- How can I explain to my son why sunlight will hurt his eyes? 
- Why does hot air rise in a column instead of cold air pressing down? 
- How to objectively counter "You are the first one to complain about that."? 
- What is the shape of the focal plane? 
- How to print "Spring" if in month is in Jan-J


In [6]:
generated_text = generate_text_with_chat('Mr. Banks was hospitalized in Fredo-clinic on Friday last week due to conjunctivitis')
print(generated_text.choices[0].message.content)


Mr. Banks is a person, Fredo clinic is a healthcare facility, and Friday last week is a date reference.
